In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.notebook import trange, tqdm, tqdm_notebook
from joblib import Parallel, delayed
import re
import h5py
import tifffile as tiff


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_processed = data_dir / 'processed'
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\Sensitivity experiment\18Oct23_multicolor pairs'


In [4]:
import os
 
os.environ['JDK_HOME'] = "C:\Program Files\Java\jdk-17"
os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-17"

os.environ['PATH'] += ';C:\\Program Files\\Java\\jdk-17\\jre\\bin\\server\\;C:\\Program Files\\Java\\jdk-17\\bin\\;C:\\Program Files\\Java\\jdk-17\\bin\\jar.exe'

# Get info

In [10]:
markers_map = {
    'cycle1': {
        1: 'DNA', 
        3: 'Sox2/Oct4',
        4: 'p-P90rsk/NF-Kb',
    },
    'cycle2': {
        1: 'DNA', 
        3: 'Cyclin D1/CDK4',
        4: 'Bim/Tom20',
    },
    'cycle3': {
        1: 'DNA', 
        2: 'Concanavalin A',
        3: 'Phalloidin',
        4: 'WGA'
    },
}

def get_info(data_raw, marker_dict):
    conditions = []
    fovs = []
    cycles = []
    channels = []
    markers = []
    paths = [] 
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        if 'after nuclease' in dirpath or 'Defocused' in dirpath or 'wrong' in dirpath:
            continue
        
            
        for name in sorted(filenames):
            if "tif" in name and "sti" in name \
            and 'overlay' not in name \
            and 'Composite' not in name:
                # Get information from image name
                row = filenames[-1][1] 
                if row == 'B':
                    condition = 'Control'
                elif row == 'C':
                    condition = '100nM'
                elif row == 'D':
                    condition = '25nM'
                elif row == 'E':
                    condition = '10nM'
                elif row == 'F':
                    condition = '5nM'
                d_split = dirpath.split('\\')
                
                well = d_split[-2].split('_')[2].split(' ')[0]
                n_split = name.split('_')
                ch = int(n_split[-1][-5])
                
                cycle = well
                try: marker = marker_dict[cycle][ch] 
                except: continue
        
                conditions.append(condition)
                fovs.append(row)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Condition": conditions,
            "FOV": fovs,
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

In [39]:
df_meta_path = data_dir / 'sensitivity_v2' / 'metadata' / 'info.csv'
df_meta_path.parent.mkdir(parents=True, exist_ok=True)


df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Created df


In [40]:
df.groupby('Condition').size()


Condition
100nM      10
10nM       10
25nM       10
5nM        10
Control    10
dtype: int64

# Save HDF5

In [36]:
import h5py

def save_hdf5(
    path: str, name: str, data: np.ndarray, attr_dict=None, mode: str = "a"
) -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        max_shape = (data_shape[0],) + data_shape[1:]
        dset = hf.create_dataset(
            name,
            shape=data_shape,
            maxshape=max_shape,
            chunks=True,
            dtype=data_type,
            compression="gzip",
        )
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f"Dataset {name} exists")

    hf.close()

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

In [48]:
df_imgs_path = data_dir / 'sensitivity_v2' / 'metadata' / 'imgs.csv'
df_imgs_path.parent.mkdir(parents=True, exist_ok=True)

temp_path = data_dir  /'sensitivity_v2' /  'hdf5' / 'raw'
temp_path.mkdir(parents=True, exist_ok=True)

df_exist = df_imgs_path.is_file()


if not df_exist:
    print('Created df')
    
    group = df.groupby(['Condition', 'FOV'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        group_cycle = df_group.groupby('Cycle')
        for cycle, df_cycle in group_cycle:
            channels = df_cycle.Channels.to_list()
            markers = df_cycle.Markers.to_list()
            paths = df_cycle.Path.to_numpy()
    
            imgs = joblib_loop(read_img, paths)
            imgs = np.array(imgs)
            info = {"Channels": channels, "Markers": markers}
            
            # hdf5 as Channel -> Z mapping
            save_hdf5(file_path, cycle, imgs, info)
    df_imgs = pd.DataFrame(rows, columns=['Condition', 'FOV', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Created df


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\thu71\Anaconda3\envs\PLA2\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [51]:
df_imgs

,Condition,FOV,Path
0,100nM,C,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
1,10nM,E,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
2,25nM,D,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
3,5nM,F,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
4,Control,B,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...


# Save Tiffile

In [52]:
from skimage import exposure, util

def contrast_str(img, n_min=0.1, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_rescale

def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img.shape[1:] for img in imgs])
    min_x, min_y = shapes.min(axis=0)
    imgs_cropped = [img[:, :min_x, :min_y] for img in imgs]
    # imgs_cropped[0] = contrast_str(imgs_cropped[0])
    return imgs_cropped


In [53]:
save_path = data_dir /  'sensitivity_v2' /'imgs' / 'raw'
save_path.mkdir(parents=True, exist_ok=True)

group = df_imgs.groupby(['Condition', 'FOV'])
for name, df_group in group:
    path = df_group.iloc[0].Path
             
    # Read images
    cycles = []
    imgs_all = []
    channels = []
    with h5py.File(path, "r") as f:
        for k in tqdm(f.keys(), total=len(f.keys()), leave=False):
            cycle = k.split('_')[0]
            channel = f[k].attrs['Channels']

            imgs = f[k][:]
            cycles.append(cycle)
            channels.append(channel)
            imgs[0] = contrast_str(imgs[0])
            imgs_all.append(imgs)
    
    imgs_same_shape = make_imgs_same_dim(imgs_all)
    
    for i, imgs in enumerate(imgs_same_shape):
        temp_path = save_path / '_'.join(np.array(name).astype(str))
        temp_path.mkdir(parents=True, exist_ok=True)

        file_name = f'001_{cycles[i]}.tif'
        file_path = temp_path / file_name

        # Write image
        tiff.imwrite(file_path, imgs)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

# Registration 

In [5]:
from ashlar import fileseries, thumbnail,reg
import matplotlib.pyplot as plt
from ashlar.scripts.ashlar import process_axis_flip

ImportError: DLL load failed while importing jnius: The specified module could not be found.

In [7]:
# Loop all images
imgs_dir = data_dir / 'sensitivity_v2' /'imgs' / 'raw'
save_dir = data_dir / 'sensitivity_v2' /'imgs' / 'registered'
save_dir.mkdir(parents=True, exist_ok=True)

imgs_dir_list = os.listdir(imgs_dir)

for dir_path in tqdm(imgs_dir_list):
    
    # Create reader for each cycle
    readers = []
    for i in range(1, 4):
        reader = fileseries.FileSeriesReader(
            str(imgs_dir / dir_path),
            pattern='{series}_'+f'cycle{i}.tif',
            overlap=0.29,
            width=1,
            height=1,
            layout='snake',
            direction='horizontal',
            pixel_size=0.18872, 
        )
        readers.append(reader)
    reader_1 = readers[0]
    
    # Run stitching
    aligner0 = reg.EdgeAligner(reader_1, channel=0, filter_sigma=2, verbose=False,)
    aligner0.run()
    
    # Generate merge image for 1 cycle
    # Parramter
    mosaic_args = {}
    mosaic_args['verbose'] = False

    mosaic = reg.Mosaic(
            aligner0,aligner0.mosaic_shape,**mosaic_args
        )
    writer_class = reg.TiffListWriter
    writer = writer_class(
            [mosaic], str(save_dir / (dir_path + '_cycle1_ch{channel}.ome.tif'))
    )
    writer.run()
    
    # Loop through rest of cycles
    aligners = list()
    aligners.append(aligner0)

    for j in range(1, len(readers)):
        aligners.append(
            reg.LayerAligner(readers[j], aligners[0], channel=0, filter_sigma=2, verbose=False)
        )
        aligners[j].run()
        mosaic = reg.Mosaic(
            aligners[j], aligners[0].mosaic_shape,**mosaic_args
        )
        writer = writer_class(
                [mosaic], str(save_dir / (dir_path +'_cycle'+str(j+1)+'_ch{channel}.ome.tif'))
        )
        writer.run()
    

  0%|          | 0/5 [00:00<?, ?it/s]

    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [0. 5.]
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [  4. -15.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [-14.   2.]
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [19.  1.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [-18. -24.]
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [ 14. -17.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [-6. -6.]
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [17. 11.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [  1. -22.]
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [ 23. -25.]


# Save data

In [10]:
def get_info(data_raw, marker_dict):
    conditions = []
    fovs = []
    cycles = []
    channels = []
    markers = []
    paths = [] 

    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name:
                # Get information from image name
                n_split = name.split('_')
                                
                cond=n_split[0]
                fov=n_split[1]
                cycle=n_split[2]
                ch = n_split[3][:3]
                try:
                    marker = marker_dict[cycle][ch]
                except:
                    continue 
                    
                conditions.append(cond)
                fovs.append(fov)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Condition": conditions,
            "FOV": fovs,
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def get_min(imgs):
    shapes = np.array([np.array(img.shape) for img in imgs])
    return np.min(shapes, axis=0)

In [31]:
from collections import defaultdict

markers_map_new = defaultdict(dict)
for k,v in markers_map.items():
    for i, (ch,marker) in enumerate(v.items()):
        markers_map_new[k][f'ch{i}'] = marker
        

In [34]:
data_raw = data_dir  / 'sensitivity_v2' /'imgs' / 'registered'
df_meta_path = data_dir /  'sensitivity_v2' / 'metadata' / 'info_sti.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map_new)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Folder is already there
Created df


In [7]:
df_imgs_path = data_dir / 'sensitivity_v2' /'metadata' / 'imgs_reg.csv'

temp_path =data_dir /  'sensitivity_v2' /'hdf5' / 'registered'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    group = df.groupby(['Condition','FOV'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        # if file_path.exists():
        #     continue
        
        channels = df_group.Channels.to_list()
        cycles = df_group.Cycle.to_list()
        markers = df_group.Markers.to_list()
        paths = df_group.Path.to_numpy()
            
        imgs = joblib_loop(read_img, paths)
        min_shape = get_min(imgs)
        imgs_cropped = np.array([img[:min_shape[0], :min_shape[1]] for img in imgs])
        info = {"Cycle": cycles, "Channel": channels, "Marker": markers}
            
            # hdf5 as Channel -> Z mapping
        save_hdf5(file_path, 'imgs', imgs_cropped, info)
    df_imgs = pd.DataFrame(rows, columns=['Condition', 'FOV', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Folder is already there
Loaded df


In [8]:
df_imgs

,Condition,FOV,Path
0,100nM,C,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
1,10nM,E,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
2,25nM,D,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
3,5nM,F,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
4,Control,B,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...


# Segmentation

In [9]:
from skimage.exposure import match_histograms, rescale_intensity
import cv2
from skimage import img_as_ubyte
import tensorflow as tf 
from skimage import exposure, util, filters, restoration

def random_crop(image, NEW_IMG_HEIGHT, NEW_IMG_WIDTH):
    cropped_image = tf.image.random_crop(
      image, size=[3, NEW_IMG_HEIGHT, NEW_IMG_WIDTH])

    return cropped_image

def make_color_overlay(input_data):
    """Create a color overlay from 2 channel image data
    
    Args:
        input_data: stack of input images
    
    Returns:
        numpy.array: color-adjusted stack of overlays in RGB mode
    """
    RGB_data = np.zeros(input_data.shape[:3] + (3, ), dtype='float32')
    
    # rescale channels to aid plotting
    for img in range(input_data.shape[0]):
        for channel in range(input_data.shape[-1]):
            # get histogram for non-zero pixels
            percentiles = np.percentile(input_data[img, :, :, channel][input_data[img, :, :, channel] > 0],
                                            [0, 100])
            rescaled_intensity = rescale_intensity(input_data[img, :, :, channel],
                                                       in_range=(percentiles[0], percentiles[1]),
                                                       out_range='float32')
            RGB_data[img, :, :, channel + 1] = rescaled_intensity
        
    # create a blank array for red channel
    return RGB_data

def contrast_str(img, n_min=10, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

In [10]:
for row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
    path = row.Path
    
    # Read images
    with h5py.File(path, "r") as f:
        
        markers = f['imgs'].attrs['Marker']
        imgs = f['imgs'][:]
    break

  0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
import napari 

napari.view_image(np.stack(imgs), name=markers, channel_axis=0, contrast_limits=[0,1])

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1734.0, 2320.0), zoom=0.3623090227731334, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 3469.0, 1.0), (0.0, 4641.0, 1.0)), current_step=(1734, 2320), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'DNA' at 0x21603017790>, <Image layer 'Sox2/Oct4' at 0x21600272200>, <Image layer 'p-P90rsk/NF-Kb' at 0x216030c3dc0>, <Image layer 'DNA [1]' at 0x2160c350f40>, <Image layer 'Cyclin D1/CDK4' at 0x2160c3bbfd0>, <Image layer 'Bim/Tom20' at 0x2160c4953f0>, <Image layer 'DNA [2]' at 0x2160c4f3e80>, <Image layer 'Concanavalin A' at 0x2160f291750>, <Image layer 'Phalloidin' at 0x2160f31c6d0>, <Image layer 'WGA' at 0x2162bed5ab0>], scale_bar=ScaleBar(v

In [14]:
df_imgs['FOV'] = df_imgs['FOV'].astype(str)

In [18]:
# Define markers to use for cytosolic segmentation
cyto_markers = ['Concanavalin A', 'Phalloidin']

# Define folder and create if don't exsit
whole_seg_path = data_dir / 'sensitivity_v2' / 'imgs' / 'segmentation'
whole_seg_path.mkdir(parents=True, exist_ok=True)

# Save combined images
for row in df_imgs.itertuples():
    # Read image
    path = row.Path
    with h5py.File(path, "r") as f:
        imgs = f['imgs'][:]
        markers = f['imgs'].attrs['Marker']

    # Get dapi and cyto imgaes
    indices = np.isin(markers, cyto_markers)
    img_dapi = imgs[-4]
    imgs_cyto = imgs[indices,:]
    
    # Contrast streching and combine to rgb image
    img_dapi = contrast_str(img_dapi, n_max=99.99)
        
    imgs_cyto_scaled = []
    for img in imgs_cyto:
        imgs_cyto_scaled = [contrast_str(img, n_max=99.9)]
    img_cyto = np.max(np.array(imgs_cyto_scaled), axis=0)
    img_rgb = np.stack([np.zeros(img_dapi.shape),img_cyto, img_dapi], axis=0).astype(np.uint8)
    
    # Crop and save
    file_name = f'{"_".join(row[1:3])}.tif'
    file_path = whole_seg_path / file_name
    tiff.imwrite(file_path, img_rgb)



# Segmentation

In [8]:
from skimage import exposure, util
from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

def contrast_str(img, n_min=0.01, n_max=99.95):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

c:\Users\thu71\AppData\Local\anaconda3\envs\cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> GPU activated? YES


In [9]:
whole_seg_path = data_dir / 'sensitivity_v2' / 'imgs' / 'segmentation'
mask_path = data_dir  / 'sensitivity_v2' / 'imgs' / 'masks'

mask_path.mkdir(parents=True, exist_ok=True)

In [10]:
# Cyto segmentaion
masks = []
for p in os.listdir(whole_seg_path):
    img = skimage.io.imread(whole_seg_path / p).transpose((2,0,1))
    
    # Cyto segmentation
    model = models.CellposeModel(gpu=True, model_type='CP')
    mask_cyto, flows, styles = model.eval(img, 
                                  channels=[2,3],
                                  diameter=250,
                                  flow_threshold=0.8,
                                  cellprob_threshold=-3
                                  )
    
    file_path = mask_path / p
    tiff.imwrite(file_path, mask_cyto)
    
    # Nuclei segemtnation
    model = models.CellposeModel(gpu=True, model_type='nuclei')
    mask_nuclei, flows, styles = model.eval(img, 
                                  channels=[3,3],
                                  diameter=80,
                                  flow_threshold=0.4,
                                  )
    
        
    file_path = mask_path / f'Nuclei_{p}'
    tiff.imwrite(file_path, mask_nuclei)


OSError: Could not find a backend to open `\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\23_PLA_revision\data\sensitivity_v2\imgs\segmentation\Thumbs.db`` with iomode `r`.